In [1]:
%matplotlib inline
import requests as http
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import lib.lastfm as lastfm
from lib.spotify import User
import time
import calendar
import datetime
import re
import unicodedata
import importlib

In [2]:
token = 'BQCq1OpPi1ylwAsTq4iGgncdu5WlD1RqikPmOcCnb0tPaz9KFplOriJvmrBWgs9xNKJW0isPowBaWMScrDTP_UQmcZWFxUb--2aHKqH65jGXcq3sSiw_K51YhVYyYAsiqpMSSBArU1lqBBD9RJrXujedUREnDbEiKEc8Ff31lu-GBho8sEv0Tx2FxDP6KouqNt3s3PU38cCZ'
received_features = ['danceability', 'energy', 'acousticness', 'valence', 'tempo']
test_user = User(received_features, 40, token=token)

Jewels & Dreams
Sultan + Shepard 
Meg & Dia
Jenna G and Futurebound
T.I. & Playboy Tre


In [3]:
importlib.reload(lastfm)
user_lastfm = lastfm.Lastfm(name='dude0faw3', spotify=test_user.library)

Starting! There are 162 pages
Page 25
Page 50
Page 75
Page 100
Page 125
Page 150


In [4]:
importlib.reload(lastfm)
user_lastfm.get_count(test_user.library, (2016, 'winter'))

NameError: name 'np' is not defined